In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point, LineString
import geopandas as gpd
from IPython.display import Image
import selenium.webdriver
import shapely.geometry
import osmnx as ox
import pyproj
from geojson.feature import Feature, FeatureCollection
from shapely.geometry import Point, Polygon, LineString
import json


In [2]:
import os
cwd = os.getcwd()
os.chdir(os.path.join(cwd,'..\..\..'))
import ExMAS.main
import ExMAS.utils
from statistics import *
from ExMAS.utils import inData as inData

In [3]:
params = ExMAS.utils.get_config('ExMAS/data/configs/my_config.json') # load the default 

In [4]:
params.nP = 200 # number of trips
params.simTime = 1 # hours
params.city = 'Amsterdam, Netherlands'
params.shared_discount = 0.3 # relative discount for sharing (i.e. instead of 10EUR single you pay (1-X)*10EUR
params.price = 1.5 # eur/km
params.VoT = 0.0035 # Value-of-time [eur/s]
params.max_degree = 5 # capacity of vehicles (paxes)

In [5]:
inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph


In [6]:
demand = [200, 400, 600, 800, 1000, 1200, 1400, 1600, 2000, 2500, 3000]
outs   = ['mean_lambda_r', 'vehhour', 'vehhour_ns', 'passhour', 'passhour_ns','passUtil',\
          'passUtil_ns' ,'shared_ratio','avg_degree']

mux = pd.MultiIndex.from_product([demand, outs])
#label = ['Discount']
#label.append(str(demand))
price  = [0.15, 0.2, 0.25,0.3,0.35]
reps   = np.arange(5)
df_    = pd.DataFrame(index=price, columns=mux)
df_    = df_.fillna(0) # with 0s rather than NaNs

stat_col = ['price','replication', 'mean_lambda_r', 'vehhour', 'vehhour_ns', 'passhour',\
           'passhour_ns','passUtil','passUtil_ns' ,'shared_ratio']
mux_stat = pd.MultiIndex.from_product([demand,stat_col])
stat_dex = np.arange(len(reps)*len(price))
df_stat = pd.DataFrame(index = stat_dex,columns=mux_stat)

In [7]:

for i in demand:
    params.nP = i # number of trips
    print('####')
    print('demand:', params.nP)
    ind = 0
    if i==3000:
        price = [0.15,0.2,0.25,0.3]
    for p in price:
        print('discount:', p)
        print('###')
        params.shared_discount = p
        mean_lambda_r = []
        vehhour     =   []
        vehhour_ns  =   []
        passhour    =   []
        passhour_ns =   []
        passutil    =   []
        passutil_ns =   []
        shared_ratio=   []
        avg_degree  =   []
        for j in range(len(reps)):
            inData = ExMAS.utils.load_albatross_csv(inData, params, sample=True)
            inData = ExMAS.main(inData, params, plot = False)
            mean_lambda_r.append(inData.sblts.res.mean_lambda)
            vehhour.append(inData.sblts.res.VehHourTrav)
            vehhour_ns.append(inData.sblts.res.VehHourTrav_ns)
            passhour.append(inData.sblts.res.PassHourTrav)
            passhour_ns.append(inData.sblts.res.PassHourTrav_ns)
            passutil.append(inData.sblts.res.PassUtility)
            passutil_ns.append(inData.sblts.res.PassUtility_ns)
            shared_ratio.append(inData.sblts.res.shared_ratio)
            avg_degree.append((inData.sblts.res.SINGLE*1+inData.sblts.res.PAIRS*2+inData.sblts.res.TRIPLES*3\
              +inData.sblts.res.QUADRIPLES*4+inData.sblts.res.QUINTETS*5)/(inData.sblts.res.SINGLE+inData.sblts.res.PAIRS+inData.sblts.res.TRIPLES\
              +inData.sblts.res.QUADRIPLES+inData.sblts.res.QUINTETS))
            
            df_stat[i,'price'].loc[ind] = p
            df_stat[i,'replication'].loc[ind] = j
            df_stat[i,'mean_lambda_r'].loc[ind] = inData.sblts.res.mean_lambda
            df_stat[i,'vehhour'].loc[ind] = inData.sblts.res.VehHourTrav
            df_stat[i,'vehhour_ns'].loc[ind] = inData.sblts.res.VehHourTrav_ns
            df_stat[i,'passhour'].loc[ind] = inData.sblts.res.PassHourTrav
            df_stat[i,'passhour_ns'].loc[ind] = inData.sblts.res.PassHourTrav_ns
            df_stat[i,'passUtil'].loc[ind] = inData.sblts.res.PassUtility
            df_stat[i,'passUtil_ns'].loc[ind] = inData.sblts.res.PassUtility_ns
            df_stat[i,'shared_ratio'].loc[ind] = inData.sblts.res.shared_ratio
            
            ind = ind + 1
            
        df_[i,'mean_lambda_r'].loc[p] = mean(mean_lambda_r)
        df_[i,'vehhour'].loc[p] = mean(vehhour)
        df_[i,'vehhour_ns'].loc[p] = mean(vehhour_ns)
        df_[i,'passhour'].loc[p] = mean(passhour)
        df_[i,'passhour_ns'].loc[p] = mean(passhour_ns)
        df_[i,'passUtil'].loc[p] = mean(passutil)
        df_[i,'passUtil_ns'].loc[p] = mean(passutil_ns)
        df_[i,'shared_ratio'].loc[p] = mean(shared_ratio)
        df_[i,'avg_degree'].loc[p]   = mean(avg_degree)

####
demand: 200
discount: 0.15
###
17-12-20 01:19:01-INFO-Initializing pairwise trip shareability between 200 and 200 trips.
17-12-20 01:19:01-INFO-creating combinations
17-12-20 01:19:02-INFO-39800	 nR*(nR-1)
17-12-20 01:19:02-INFO-Reduction of feasible pairs by 99.99%
17-12-20 01:19:02-INFO-Degree 2 	Completed
17-12-20 01:19:02-INFO-trips to extend at degree 2 : 5
17-12-20 01:19:02-INFO-At degree 2 feasible extensions found out of 0 searched
17-12-20 01:19:02-INFO-Degree 3 	Completed
17-12-20 01:19:02-INFO-No more trips to exted at degree 3
17-12-20 01:19:02-INFO-Matching 200 trips to 205 rides in order to min u_veh
17-12-20 01:19:02-INFO-Problem solution: Optimal. 
Total costs for single trips:         95,068 
reduced by matching to:               94,319
17-12-20 01:19:02-INFO-Calculations  completed
17-12-20 01:19:02-INFO-DotMap(VehHourTrav=94259, VehHourTrav_ns=95068, PassHourTrav=95651, PassHourTrav_ns=95068, PassUtility=1473.755825, PassUtility_ns=1474.604, mean_lambda=0.129093

17-12-20 01:19:17-INFO-Calculations  completed
17-12-20 01:19:17-INFO-DotMap(VehHourTrav=87345, VehHourTrav_ns=90628, PassHourTrav=91764, PassHourTrav_ns=90628, PassUtility=1396.0292, PassUtility_ns=1405.883, mean_lambda=0.1844903811685984, revenue_s=108753.6, revenue_ns=135942.0, Fare_Discount=-0.19999999999999996, nR=200, SINGLE=174, PAIRS=13, TRIPLES=0, QUADRIPLES=0, QUINTETS=0, PLUS5=0, shared_ratio=0.13, fleet_size_nonshared=21.0, fleet_size_shared=20.0, lambda_shared=0.1844903811685984)
17-12-20 01:19:19-INFO-Initializing pairwise trip shareability between 200 and 200 trips.
17-12-20 01:19:19-INFO-creating combinations
17-12-20 01:19:19-INFO-39800	 nR*(nR-1)
17-12-20 01:19:19-INFO-Reduction of feasible pairs by 99.97%
17-12-20 01:19:19-INFO-Degree 2 	Completed
17-12-20 01:19:19-INFO-trips to extend at degree 2 : 31
17-12-20 01:19:19-INFO-At degree 2 feasible extensions found out of 0 searched
17-12-20 01:19:19-INFO-Degree 3 	Completed
17-12-20 01:19:19-INFO-No more trips to exted

17-12-20 01:19:33-INFO-Calculations  completed
17-12-20 01:19:33-INFO-DotMap(VehHourTrav=89668, VehHourTrav_ns=95459, PassHourTrav=99861, PassHourTrav_ns=95459, PassUtility=1456.7805250000001, PassUtility_ns=1480.5595, mean_lambda=0.17811524725716676, revenue_s=107391.375, revenue_ns=143188.5, Fare_Discount=-0.25, nR=200, SINGLE=159, PAIRS=17, TRIPLES=1, QUADRIPLES=1, QUINTETS=0, PLUS5=0, shared_ratio=0.20499999999999996, fleet_size_nonshared=29.0, fleet_size_shared=28.0, lambda_shared=0.17811524725716676)
17-12-20 01:19:35-INFO-Initializing pairwise trip shareability between 200 and 200 trips.
17-12-20 01:19:35-INFO-creating combinations
17-12-20 01:19:35-INFO-39800	 nR*(nR-1)
17-12-20 01:19:36-INFO-Reduction of feasible pairs by 99.88%
17-12-20 01:19:36-INFO-Degree 2 	Completed
17-12-20 01:19:36-INFO-trips to extend at degree 2 : 130
17-12-20 01:19:36-INFO-At degree 2 feasible extensions found out of 0 searched
17-12-20 01:19:36-INFO-Degree 3 	Completed
17-12-20 01:19:36-INFO-No more

17-12-20 01:19:52-INFO-Calculations  completed
17-12-20 01:19:52-INFO-DotMap(VehHourTrav=92246, VehHourTrav_ns=102995, PassHourTrav=112168, PassHourTrav_ns=102995, PassUtility=1534.811475, PassUtility_ns=1597.4605, mean_lambda=0.17606581125827814, revenue_s=108144.75, revenue_ns=154492.5, Fare_Discount=-0.3, nR=200, SINGLE=122, PAIRS=36, TRIPLES=2, QUADRIPLES=0, QUINTETS=0, PLUS5=0, shared_ratio=0.39, fleet_size_nonshared=33.0, fleet_size_shared=26.0, lambda_shared=0.17606581125827814)
17-12-20 01:19:54-INFO-Initializing pairwise trip shareability between 200 and 200 trips.
17-12-20 01:19:54-INFO-creating combinations
17-12-20 01:19:54-INFO-39800	 nR*(nR-1)
17-12-20 01:19:54-INFO-Reduction of feasible pairs by 99.77%
17-12-20 01:19:54-INFO-Degree 2 	Completed
17-12-20 01:19:54-INFO-trips to extend at degree 2 : 159
17-12-20 01:19:54-INFO-At degree 2 feasible extensions found out of 3 searched
17-12-20 01:19:54-INFO-Degree 3 	Completed
17-12-20 01:19:54-INFO-trips to extend at degree 3 

17-12-20 01:20:11-INFO-Degree 4 	Completed
17-12-20 01:20:11-INFO-No more trips to exted at degree 4
17-12-20 01:20:12-INFO-Matching 200 trips to 533 rides in order to min u_veh
17-12-20 01:20:12-INFO-Problem solution: Optimal. 
Total costs for single trips:         95,211 
reduced by matching to:               87,527
17-12-20 01:20:12-INFO-Calculations  completed
17-12-20 01:20:12-INFO-DotMap(VehHourTrav=87182, VehHourTrav_ns=95211, PassHourTrav=104559, PassHourTrav_ns=95211, PassUtility=1424.7119000000002, PassUtility_ns=1476.864, mean_lambda=0.18049846139390668, revenue_s=92830.725, revenue_ns=142816.5, Fare_Discount=-0.35, nR=200, SINGLE=136, PAIRS=23, TRIPLES=6, QUADRIPLES=0, QUINTETS=0, PLUS5=0, shared_ratio=0.31999999999999995, fleet_size_nonshared=25.0, fleet_size_shared=19.0, lambda_shared=0.18049846139390668)
####
demand: 400
discount: 0.15
###
17-12-20 01:20:14-INFO-Initializing pairwise trip shareability between 400 and 400 trips.
17-12-20 01:20:14-INFO-creating combination

...

17-12-20 07:13:16-INFO-Initializing pairwise trip shareability between 2500 and 2500 trips.
17-12-20 07:13:16-INFO-creating combinations
17-12-20 07:13:24-INFO-6247500	 nR*(nR-1)
17-12-20 07:15:21-INFO-Reduction of feasible pairs by 99.60%
17-12-20 07:15:21-INFO-Degree 2 	Completed
17-12-20 07:15:22-INFO-trips to extend at degree 2 : 49458
17-12-20 07:19:13-INFO-At degree 2 feasible extensions found out of 48844 searched
17-12-20 07:19:13-INFO-Degree 3 	Completed
17-12-20 07:19:13-INFO-trips to extend at degree 3 : 48844
17-12-20 07:23:18-INFO-At degree 3 feasible extensions found out of 22240 searched
17-12-20 07:23:19-INFO-Degree 4 	Completed
17-12-20 07:23:19-INFO-trips to extend at degree 4 : 22240
17-12-20 07:24:51-INFO-At degree 4 feasible extensions found out of 3412 searched
17-12-20 07:24:51-INFO-Degree 5 	Completed
17-12-20 07:24:51-INFO-Max degree reached 5
17-12-20 07:24:51-INFO-Trips still possible to extend at degree 5 : 3412
17-12-20 07:27:49-INFO-Matching 2500 trips to 

17-12-20 08:32:21-INFO-Degree 3 	Completed
17-12-20 08:32:21-INFO-No more trips to exted at degree 3
17-12-20 08:32:23-INFO-Matching 3000 trips to 4299 rides in order to min u_veh
17-12-20 08:32:30-INFO-Problem solution: Optimal. 
Total costs for single trips:      1,437,682 
reduced by matching to:            1,319,861
17-12-20 08:32:31-INFO-Calculations  completed
17-12-20 08:32:31-INFO-DotMap(VehHourTrav=1314176, VehHourTrav_ns=1437682, PassHourTrav=1469272, PassHourTrav_ns=1437682, PassUtility=22148.123025, PassUtility_ns=22299.749, mean_lambda=0.21493436366687824, revenue_s=1833044.55, revenue_ns=2156523.0, Fare_Discount=-0.14999999999999997, nR=3000, SINGLE=2242, PAIRS=379, TRIPLES=0, QUADRIPLES=0, QUINTETS=0, PLUS5=0, shared_ratio=0.2526666666666667, fleet_size_nonshared=387.0, fleet_size_shared=352.0, lambda_shared=0.21493436366687824)
17-12-20 08:32:34-INFO-Initializing pairwise trip shareability between 3000 and 3000 trips.
17-12-20 08:32:34-INFO-creating combinations
17-12-2

17-12-20 08:44:41-INFO-Problem solution: Optimal. 
Total costs for single trips:      1,426,699 
reduced by matching to:            1,184,773
17-12-20 08:44:42-INFO-Calculations  completed
17-12-20 08:44:42-INFO-DotMap(VehHourTrav=1175323, VehHourTrav_ns=1426699, PassHourTrav=1516892, PassHourTrav_ns=1426699, PassUtility=21630.677775000004, PassUtility_ns=22129.7165, mean_lambda=0.25772121800935754, revenue_s=1712038.8, revenue_ns=2140048.5, Fare_Discount=-0.19999999999999998, nR=3000, SINGLE=1572, PAIRS=630, TRIPLES=52, QUADRIPLES=3, QUINTETS=0, PLUS5=0, shared_ratio=0.476, fleet_size_nonshared=380.0, fleet_size_shared=317.0, lambda_shared=0.25772121800935754)
17-12-20 08:44:45-INFO-Initializing pairwise trip shareability between 3000 and 3000 trips.
17-12-20 08:44:45-INFO-creating combinations
17-12-20 08:44:55-INFO-8997000	 nR*(nR-1)
17-12-20 08:46:44-INFO-Reduction of feasible pairs by 99.97%
17-12-20 08:46:45-INFO-Degree 2 	Completed
17-12-20 08:46:45-INFO-trips to extend at degre

17-12-20 09:03:25-INFO-Degree 5 	Completed
17-12-20 09:03:25-INFO-Max degree reached 5
17-12-20 09:03:25-INFO-Trips still possible to extend at degree 5 : 16
17-12-20 09:03:52-INFO-Matching 3000 trips to 26525 rides in order to min u_veh
17-12-20 09:05:06-INFO-Problem solution: Optimal. 
Total costs for single trips:      1,445,862 
reduced by matching to:            1,099,881
17-12-20 09:05:08-INFO-Calculations  completed
17-12-20 09:05:08-INFO-DotMap(VehHourTrav=1090956, VehHourTrav_ns=1445862, PassHourTrav=1604512, PassHourTrav_ns=1445862, PassUtility=21591.449225, PassUtility_ns=22426.705500000004, mean_lambda=0.3054828350717085, revenue_s=1626594.75, revenue_ns=2168793.0, Fare_Discount=-0.25, nR=3000, SINGLE=1172, PAIRS=595, TRIPLES=175, QUADRIPLES=27, QUINTETS=1, PLUS5=0, shared_ratio=0.6093333333333333, fleet_size_nonshared=414.0, fleet_size_shared=301.0, lambda_shared=0.3054828350717085)
17-12-20 09:05:11-INFO-Initializing pairwise trip shareability between 3000 and 3000 trips.

17-12-20 09:51:15-INFO-At degree 3 feasible extensions found out of 7018 searched
17-12-20 09:51:15-INFO-Degree 4 	Completed
17-12-20 09:51:15-INFO-trips to extend at degree 4 : 7018
17-12-20 09:51:36-INFO-At degree 4 feasible extensions found out of 704 searched
17-12-20 09:51:36-INFO-Degree 5 	Completed
17-12-20 09:51:36-INFO-Max degree reached 5
17-12-20 09:51:36-INFO-Trips still possible to extend at degree 5 : 704
17-12-20 09:53:10-INFO-Matching 3000 trips to 73520 rides in order to min u_veh
17-12-20 09:57:01-INFO-Problem solution: Optimal. 
Total costs for single trips:      1,414,967 
reduced by matching to:            1,004,685
17-12-20 09:57:03-INFO-Calculations  completed
17-12-20 09:57:03-INFO-DotMap(VehHourTrav=997620, VehHourTrav_ns=1414967, PassHourTrav=1644601, PassHourTrav_ns=1414967, PassUtility=20849.48215, PassUtility_ns=21947.797, mean_lambda=0.34782571835986575, revenue_s=1485715.3499999999, revenue_ns=2122450.5, Fare_Discount=-0.30000000000000004, nR=3000, SINGLE

In [8]:
df_.to_csv (r"C:\Users\mmari\Desktop\MARKO STUFF\UNI\MSc\2nd Transport & Planning\Thesis\Project\ExMAS\ExMAS\data\big_test2.csv", index = False, header=True)

In [9]:
df_stat.to_csv (r"C:\Users\mmari\Desktop\MARKO STUFF\UNI\MSc\2nd Transport & Planning\Thesis\Project\ExMAS\ExMAS\data\big_test_stat.csv", index = False, header=True)